In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import RMSprop
from keras import regularizers

In [2]:
train_dir = '/kaggle/input/fruit-classification10-class/MY_data/train'
validation_dir = '/kaggle/input/fruit-classification10-class/MY_data/test'

In [3]:

#Data Generator

#Performing some operations for more data from a single picture
train_datagen = ImageDataGenerator(
                                    rescale=1./255,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,             #if you want to divide test and validation then add parameter: validation_split=0.2, 8-2 split
                                    fill_mode='nearest')

#We can provide same eefects to test datagen but no need as we are only testing data
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                          train_dir,               # This is the source directory for training images
                          target_size=(150, 150),  # All images will be resized to 150x150
                          batch_size=20,           # Since we use binary_crossentropy loss, we need binary labels
                          class_mode='binary')     #For categorical use categorical in class mode
                          #subset='training')      #set as training data in case you have added validation split in the datagen

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
                          validation_dir,
                          target_size=(150, 150),
                          batch_size=20,
                          class_mode='binary')
                          #subset='validation')   

Found 2301 images belonging to 10 classes.
Found 1025 images belonging to 10 classes.


In [4]:
#Model Making

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),    #Change the shape as your requirement
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Dropout(0.2),                       #Helps to avoid overfitting of model by dropping some weights
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')      #Sigmoid is used data is in binary form for categorical use softmax
])


#Compile Model

model.compile(loss='binary_crossentropy',               #For categorical use categorical)crossentropy
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])

In [5]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,  #steps = total_image_train / batch_size
      epochs=10,            #No of epochs
      validation_data=validation_generator,
      validation_steps=50)  #steps = total_image_test / batch_size

Epoch 1/10
100/100 [==============================] - 39s 380ms/step - loss: -32446208000.0000 - accuracy: 0.1030 - val_loss: -190201708544.0000 - val_accuracy: 0.1030
Epoch 2/10
100/100 [==============================] - 36s 361ms/step - loss: -1478677495808.0000 - accuracy: 0.1050 - val_loss: -4644238000128.0000 - val_accuracy: 0.1020
Epoch 3/10
100/100 [==============================] - 37s 365ms/step - loss: -14479108603904.0000 - accuracy: 0.0964 - val_loss: -33020314648576.0000 - val_accuracy: 0.1000
Epoch 4/10
100/100 [==============================] - 36s 360ms/step - loss: -71657771237376.0000 - accuracy: 0.1020 - val_loss: -140876437258240.0000 - val_accuracy: 0.1010
Epoch 5/10
100/100 [==============================] - 36s 362ms/step - loss: -251841547862016.0000 - accuracy: 0.0994 - val_loss: -440354977349632.0000 - val_accuracy: 0.1030
Epoch 6/10
100/100 [==============================] - 36s 362ms/step - loss: -697464571035648.0000 - accuracy: 0.0989 - val_loss: -11326554

In [6]:
#Save the model to use it later and to save time next time

model.save("fruitModel.keras")

In [7]:
import h5py
from tensorflow import keras

model = keras.models.load_model('/kaggle/working/fruitModel.keras')     #Give path to your model

In [8]:
from keras.preprocessing import image
import numpy as np
import os  # Add import for the os module

status = ['First', 'Second']  # Rename this as per your needs, e.g., Cat or dog, house or horse, etc.
path = '/kaggle/input/fruit-classification10-class/MY_data/predict'  # Path to the directory containing images you want to predict

# List to store predictions
predictions = []

# Iterate over images in the directory
for file_name in os.listdir(path):
    if file_name.endswith(".jpeg") or file_name.endswith(".jpg"):
        # Construct the full path to the image
        img_path = os.path.join(path, file_name)

        # Load and preprocess the image
        img = image.load_img(img_path, target_size=(150, 150))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)

        # Make predictions
        prediction = model.predict(x)
        predictions.append((file_name, prediction))

# Print or process the predictions as needed
for file_name, prediction in predictions:
    print(f"File: {file_name}, Prediction: {prediction}")
    

1/1 [==============================] - 0s 21ms/step
File: 0124.jpeg, Prediction: [[1.]]
File: 023.jpeg, Prediction: [[1.]]
File: 365.jpeg, Prediction: [[1.]]
File: 659.jpeg, Prediction: [[1.]]
File: img_241.jpeg, Prediction: [[1.]]
File: 22.jpeg, Prediction: [[1.]]
File: 9.jpeg, Prediction: [[1.]]
File: 011.jpeg, Prediction: [[1.]]
File: 0.jpeg, Prediction: [[1.]]
File: img_171.jpeg, Prediction: [[1.]]
File: img_31.jpeg, Prediction: [[1.]]
File: img_361.jpeg, Prediction: [[1.]]
File: img_41.jpeg, Prediction: [[1.]]
File: img_21.jpeg, Prediction: [[1.]]
File: img_411.jpeg, Prediction: [[1.]]
File: 4.jpeg, Prediction: [[1.]]
File: 2.jpeg, Prediction: [[1.]]
File: 33.jpeg, Prediction: [[1.]]
File: img_421.jpeg, Prediction: [[1.]]
File: 65.jpeg, Prediction: [[1.]]
File: img_371.jpeg, Prediction: [[1.]]
File: 68.jpeg, Prediction: [[1.]]
File: 99.jpeg, Prediction: [[1.]]
File: img_01.jpeg, Prediction: [[1.]]
File: 77.jpeg, Prediction: [[1.]]
File: 0122.jpeg, Prediction: [[1.]]
File: 321.jpeg